In [ ]:
import import_ipynb
import json
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.notebook import tqdm
from scipy.interpolate import interp1d
from State_simulation_Final import State
from copy import deepcopy
import random

import zmq
context = zmq.Context()

import qinfer
from qinfer import FiniteOutcomeModel
from qinfer import Distribution
from qinfer import SMCUpdater, UniformDistribution

In [ ]:
ADDR = "tcp://vidyut.physik.uni-mainz.de:55551"

socket = context.socket(zmq.REQ)
socket.connect(ADDR)

def json_rpc_request(method:str, params, id=None) -> dict:
    if id is None:
        id=1
    request={
        "jsonrpc":"2.0",
        "method":method,
        "id": id,
    }
    if params is not None:
        request["params"]=params
    return request

def parse_rpc_reply(reply:dict)->any:
    assert reply["jsonrpc"]=='2.0'
    if "error" in reply:
        raise Exception(reply["error"])
    else:
        return reply["id"], reply["result"]

def send_recv_rpc_message(method:str, params, id=None)->any:
    request = json_rpc_request(method, params, id)
    request_id=request["id"]
    socket.send_json(request)
    print("Waiting for reply...")
    reply = socket.recv_json()
    
    reply_id, result = parse_rpc_reply(reply)
    assert request_id == reply_id, "Ids of request (%s) and reply (%s) do not match."% (request_id, reply_id)
    return result

In [ ]:
interp_list = [[]]*4

psi=State([1,0,0,0,0])
d_range=np.linspace(0, 1.4, 51)

for j in range(4):
    
    psi=State([1,0,0,0,0]) #creates the state |gg>
    
    probs = psi.detune(2*j+1, Rabi=d_range, unitary=True, factor_SPAM=0.008, factor_dep=0.005)

    
    interp = interp1d(d_range, probs, kind='cubic', bounds_error=False, fill_value=0., axis=0) 
    
    interp_list[j]=deepcopy(interp)

In [ ]:
def plot(updater, i, mu, sig, optimal_setting, modelparams):
    
    # Just a plotting function
    
    plt.figure(figsize=(5, 12), dpi=150)
    
    ax = plt.subplot(411)
    ax.hist(updater.particle_locations[:,0], weights=updater.particle_weights)
    ax.set_xlabel('Rabi')

    pi4 = (optimal_setting>3).astype(int)
    
    ax = plt.subplot(412)
    steps=np.arange(i+1)
    ax.scatter(steps[:-1], 2*optimal_setting+1)


    plt.xlabel('step')
    plt.ylabel('# of gates')

    ax = plt.subplot(413)

    plt.plot(steps, mu[:,0])
    plt.legend(['$\mu_\Omega$'])
    plt.xscale('log')
    plt.grid()

    ax = plt.subplot(414)
    
    plt.plot(steps, sig[:,0]*np.sqrt(2/np.pi))
    plt.plot(steps, np.abs(mu[:,0]*np.sqrt(1/(1-0.36))-modelparams[0]/50))
    plt.legend(['$\sigma_\Omega$', '$|\mu_\Omega-\mu_{opt}|$'])
    plt.xscale('log')
    plt.yscale('log')
    plt.grid()

In [ ]:
class LSModel(FiniteOutcomeModel):
    
    @property
    def n_modelparams(self):
        # The model parameters (number of things the qubit population depends over, 
        # in this case, the drive power, SPAM and depolarizing factors)
        return 3
    
    @property
    def is_n_outcomes_constant(self):
        return True
    def n_outcomes(self, expparams):
        return 3 #gg, ee, eg+ge
    
    def are_models_valid(self, modelparams):
        
        return np.logical_and(np.logical_and(modelparams[:,0] > 0.2, modelparams[:,0] < 1.4), 
                              np.logical_and(modelparams[:,1] > 0., modelparams[:,1] < 0.02),
                              np.logical_and(modelparams[:,2] > 0., modelparams[:,2] < 0.005))
    
        # The last two just correspond to the limits of the interpolation range
        # The first one just assumes that we know that our laser power is not too low (so greater than 0.2)
        # and has to be limited above by 1.4 so that the likelihood for a single LS gate
        # in the first step is a monotonic function, that is, it will not create a multimodal
        # particle distribution already in step 2.


    @property
    def expparams_dtype(self):
        return [('n_gates', 'int')]

    def likelihood(self, outcomes, modelparams, expparams):
        
        # Computes the likelihood of an outcome given a modelparam and an expparam (# of gates)
        
        super(LSModel, self).likelihood(
            outcomes, modelparams, expparams)
        
        expparams=np.squeeze(expparams)
        probabilities = interp_list[expparams]((modelparams[:, np.newaxis, 0], 
                                                modelparams[:, np.newaxis, 1], 
                                                modelparams[:, np.newaxis, 2])).T.reshape(3, -1, 1).real
        # Computes the likelihood
        

        if len(np.shape(outcomes)) == 0:
            outcomes = np.array(outcomes)[None]
                    
        likelihood_array= np.concatenate([
            probabilities[np.newaxis, outcomes[idx]]
            for idx in range(outcomes.shape[0])
        ])
        # Generates the likelihood array for every outcome that you want to measure

        return likelihood_array

    
def variance(positions, weights): # Usual variance formula
    
    mean = np.dot(weights, positions)
    return np.dot(weights, (positions-mean)**2)

def expected_variance_decrease(updater, expparams):
        
        os = np.arange(3) # the possible outcomes, gg, ee, and eg/ge

        w_hyp, L, N = updater.hypothetical_update(
                os[:-1], 
                expparams, 
                return_normalization=True, 
                return_likelihood=True
            )
        
        # calculates the weights of the particles for each outcome
        
        w_hyp_last_outcome = (1 - L.sum(axis=0)) * updater.particle_weights[np.newaxis, :]
        N = np.concatenate([N[:,:,0], np.sum(w_hyp_last_outcome[np.newaxis,:,:], axis=2)], axis=0)
        w_hyp_last_outcome = w_hyp_last_outcome / N[-1,:,np.newaxis]
        w_hyp = np.concatenate([w_hyp, w_hyp_last_outcome[np.newaxis,:,:]], axis=0)

        
        sig_exp = sum(variance(updater.particle_locations, w_hyp[i,0])*N[i] for i in os)
        # calculates the expected variance after this measurement
        
        sig = variance(updater.particle_locations, updater.particle_weights)
        # the current variance of the particle distribution
        
        return (sig-sig_exp)[0]


def power2rabi(x, logpower):
    
    if logpower:
        
        # We don't now the precise relation between laser power and Rabi frequency, so here
        # we define some function that could be some power-to-Rabi relation. We choose to work here
        # with this log function because it seems to be qualitatively similar to the behavior of the power-to-Rabi
        # relation in the actual quantum computer.
        # The only requirement for this function is that at power=optimal power, Rabi=1
        
        return np.array([[np.log2(x[0]+1), x[1], x[2]]])
    else:
        return np.array([[x[0], x[1], x[2]]])
    
def measurement_loop(n_meas, n_shots_per_meas, model, updater, meas_settings, modelparams, 
                     thresh=None, plot_distributions=None, logpower=None):
    """
    
    This program takes an updater with a particle distribution that follows a uniform distribution over a
    specified range. At every step in range(n_meas) (or until it stops), it finds what is the best meas setting
    to use (how many gates to apply), simulates a measurement with that setting and with the "actual" rabi
    frequency, and with the outcome of that simulated measurement updates the position of the particles,
    reducing the variance of the distribution.
    
    
    :model: where we specify the outcomes, their likelihoods, their parameters, etc. (see LSModel above)
    :updater: the object that contains information about the particle locations and weights
    :meas_settings: whether we apply 1,3,5,7 gates, represented by an array [0,1,2,3]
    :modelparams: the "actual" values we want to obtain, for simulation purposes only, a list with a single value
    :n_meas, n_shots_per_meas: int's
    :thresh: float, the value of the particle distribution variance at which we stop the Bayesian update
    :plot_distributions: bool, in case you want to see the particle distribution and measurement outcomes at each step
    :logpower: bool, in case you want to use some power-to-Rabi relation that is not the trivial one (Rabi=power)
    
    
    """
    particle_locations=np.zeros((n_meas, 1000))
    particle_weights=np.zeros((n_meas, 1000))

    optimal_setting = np.zeros(n_meas)
    ydata = np.zeros((n_meas,n_shots_per_meas))
    mu = np.zeros((n_meas+1, model.n_modelparams))
    sig = np.zeros((n_meas+1, model.n_modelparams))
    
    mu[0] = updater.est_mean()
    sig[0] = variance(updater.particle_locations, updater.particle_weights)

    for i in tqdm(range(n_meas)):
        info_gain = [expected_variance_decrease(updater, np.array([meas_settings[i]])) 
                     for i in range(len(meas_settings))]

        opt_meas_set = np.array([meas_settings[np.argmax(info_gain)]]) 
        # Chooses the one that will minimize the variance the most
        
        shots = send_recv_rpc_message(method = "bell_state_calibration", 
                          params={"shots": n_shots_per_meas, "gates": int(2*np.argmax(info_gain)+1),
                                  "global_vars":{"Glightforce.lightforce_s::WobblePower": modelparams[0]}})

        gg = shots[0]*n_shots_per_meas
        ee = shots[3]*n_shots_per_meas

        datum = [1]*int(ee)+[0]*int(gg)+[2]*(n_shots_per_meas-int(gg)-int(ee))
        datum = np.array(datum)
        random.shuffle(datum) # Generates individual shot data from received qubit population data 
        
        
        if plot_distributions:
            fig, ax = plt.subplots()
            ax.hist(updater.particle_locations[:,0]*mu[i,0]/mu[0,0], weights=updater.particle_weights)
            print(np.unique(datum, return_counts=True))
        
        for d in datum:
            updater.update(d, opt_meas_set)
            # updates the weights of the particles

        ydata[i] = datum
        optimal_setting[i] = opt_meas_set[0]

        mu[i+1,0] = mu[i,0]*updater.est_mean()[0]
        mu[i+1,1:] = updater.est_mean()[1:]
        
        sig[i+1] = variance(updater.particle_locations, updater.particle_weights)
        
        modelparams[0]/=updater.est_mean()[0]
        # updates the power of the next measurement, assuming that the relation between
        # Rabi and power is linear (because the optimizer doesn't "know" the power-to-Rabi relation)
        
        updater.particle_locations[:,0]/=updater.est_mean()[0]
        # updates the particle distribution, bringing the average to 0
        
        
        if thresh!=None:
            if np.all(np.sqrt(sig[i+1])<thresh):
                break # stops updating when variance reaches threshold
    
    mu[:,0]/=mu[0,0]
    updater.particle_locations[:,0]*=mu[i+1,0]

    ydata = 1-ydata.mean(axis=1)
    
    i+=1
    
    mu = mu[:i+1]
    sig = sig[:i+1]
    optimal_setting = optimal_setting[:i]
    ydata = ydata[:i+1]
    particle_locations=particle_locations[:i+1]
    particle_weights=particle_weights[:i+1]
    
    
    return updater, i, mu, sig, optimal_setting, ydata, modelparams

In [ ]:
model = LSModel()
params_0=np.copy(modelparams)

prior = UniformDistribution([[.2, 1.4]])
# We can start with a smaller range for the prior of the Rabi frequency, depending on 
# our current knowledge of possible laser powers.

updater = SMCUpdater(model, 1000, prior)
# Creates an updater with 1000 particles

meas_settings = np.arange(4).astype(int) # 0, 1, 2, 3 corresponds to 1, 3, 5, 7 gates respectively

n_meas = 10

n_shots_per_meas = 100

thresh=0.01

updater, i, mu, sig, optimal_setting, ydata, params = measurement_loop(
    n_meas, n_shots_per_meas, model, deepcopy(updater),
    meas_settings, modelparams, thresh, plot_distributions=False)

plot(updater, i, mu, np.sqrt(sig), optimal_setting, params_0)

print("Finished in {} steps with value power={:.4f}\pm{:.4f}, SPAM={:.4f}\pm{:.4f}".format(
    i, mu[-1,0], sig[-1,0]/mu[-1,0]*params[0], mu[-1,1], sig[-1,1]))    